<h1>Indice<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<a href="https://colab.research.google.com/github/gionatamassibenincasa/statistica_descrittiva/blob/master/AnalisiDatiCovid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly as py
import plotly.graph_objects as go
from plotly.offline import iplot
import re
import numpy as np

df_nazione = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv")
df_regioni = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv")
df_marche = df_regioni[df_regioni['denominazione_regione'] == "Marche"] # df_regioni[df_regioni['codice_regione'] == 11]
df_puglia = df_regioni[df_regioni['denominazione_regione'] == "Puglia"]

nomi_campi = ["totale_positivi", "totale_ospedalizzati", "terapia_intensiva", "variazione_totale_positivi", "nuovi_positivi", "deceduti"]
finestra = [6, 1, 1, 6, 6, 1]
ritardo = [3, 0, 0, 4, 4, 1]
nomi_legenda = list(map(lambda campo: re.sub("_", " ", campo).capitalize(), nomi_campi))

In [ ]:
for df in [ df_nazione, df_marche, df_puglia]:
    fig = go.Figure()
    i = 0
    while i < len(nomi_campi):
        fig.add_trace(go.Bar(
            x=df.data,
            y=getattr(df, nomi_campi[i]),
            name= nomi_legenda[i]
        ))
        fig.add_trace(go.Line(
            x=df.data,
            y=getattr(df, nomi_campi[i]).rolling(finestra[i], win_type="hamming").mean().shift(-ritardo[i]+1),
            name=("MM " +  nomi_legenda[i])
        ))
        i = i + 1
        
    # Variazione percentuale dei positivi
    fig.add_trace(go.Bar(
            x=df.data,
            y=100 * df.variazione_totale_positivi / df.totale_positivi,
            name="Var. perc. positivi"
    ))
    # Rapporto nuovi contagi su totale positivi
    fig.add_trace(go.Line(
            x=df.data,
            y=1000 * df.nuovi_positivi / df.totale_positivi,
            name="Rt per mille"
    ))
    fig.add_trace(go.Bar(
            x=df.data,
            y= np.diff(df.tamponi),
            name="Tamponi giornalieri"
    ))
    fig.add_trace(go.Bar(
            x=df.data,
            y=np.diff(df.casi_testati),
            name="Tamponi giornalieri nuove diagnosi"
    ))
    fig.add_trace(go.Bar(
            x=df.data,
            y= np.diff(df.tamponi) - np.diff(df.casi_testati),
            name="Tamponi ricerca guarigioni"
    ))
    fig.add_trace(go.Bar(
            x=df.data,
            y= np.gradient(df.deceduti),
            name="Gradiente deceduti"
    ))
    fig.add_trace(go.Bar(
            x=df.data,
            y= np.diff(df.casi_testati) / np.diff(df.tamponi),
            name="Tasso ricerca nuovi casi"
    ))
    
    fig.update_layout(barmode='group', xaxis_tickangle=-45, title_text = df['denominazione_regione'].iloc[0] if 'denominazione_regione' in df.columns else 'Italia')
    fig.show()